### Tratando outliers, mascarando dados sensíveis e normalizando os dados

In [ ]:
import os
import pandas as pd
import plotly.express as px
import numpy as np
import warnings
import json
from collections import defaultdict
from sklearn.preprocessing import MinMaxScaler

pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings("ignore")

caminho_base = "G:/Meu Drive/Estudos/Mestrado/Projeto de Mestrado/"

# Define caminhos para os dados brutos e para os dados tratados
caminho_dados_originais = os.path.join(caminho_base, "bases_originais")
caminho_dados_tratados = os.path.join(caminho_base, "bases_tratadas")

os.makedirs(caminho_dados_tratados, exist_ok=True)

# ==============================================================================
# FUNÇÕES AUXILIARES PARA LIMPEZA DE DADOS
# ==============================================================================

def corrigir_sequencias_de_zeros(df_grupo, coluna, min_sequencia=2):
    # Encontra sequências de zeros (>= min_sequencia) e as preenche usando interpolação linear.
    df = df_grupo.copy()
    is_zero = df[coluna] == 0
    blocos = (is_zero != is_zero.shift()).cumsum()
    tamanho_blocos = df.groupby(blocos)[coluna].transform('size')
    condicao = (is_zero) & (tamanho_blocos >= min_sequencia)
    df.loc[condicao, coluna] = np.nan
    df[coluna] = df[coluna].interpolate(method='linear')
    df[coluna] = df[coluna].ffill().bfill()
    return df

def filtrar_e_imputar_spikes_robusto(df_grupo, coluna, threshold_pico=1.5):
    # Corrige picos extremos e zeros isolados que sobraram.
    df = df_grupo.copy()
    prev_val = df[coluna].shift(1)
    next_val = df[coluna].shift(-1)
    media_vizinhos = (prev_val + next_val) / 2
    condicao_pico = (df[coluna] > prev_val * threshold_pico) & (df[coluna] > next_val * threshold_pico) & (prev_val > 0) & (next_val > 0)
    condicao_queda = (df[coluna] == 0) & (prev_val.notna()) & (prev_val != 0) & (next_val.notna()) & (next_val != 0)
    is_outlier = condicao_pico | condicao_queda
    df[coluna] = np.where(is_outlier, media_vizinhos, df[coluna])
    df[coluna].fillna(df_grupo[coluna], inplace=True)
    return df

# ==============================================================================
# PIPELINE PRINCIPAL DE PROCESSAMENTO DE DADOS
# ==============================================================================

print("\n--- INICIANDO PROCESSAMENTO ---")

# ETAPA 1: Carregando dados brutos
df = pd.read_csv(os.path.join(caminho_dados_originais, "Medicoes_2018-2024.csv"), sep=',', encoding='latin-1', skiprows=1)

# ETAPA 2: Gerando/carregando o mapeamento de transformadores
caminho_mapeamento = os.path.join(caminho_dados_originais, "mapeamento_trafos.json")

if not os.path.exists(caminho_mapeamento):
    trafos_originais = df['Equipamento Medição'].unique()
    grupos = defaultdict(list)
    for trafo in sorted(trafos_originais):
        sigla = trafo[:3]
        grupos[sigla].append(trafo)
    dicionario_mascarado = {}
    contador_t = 1
    alfabeto = 'abcdefghijklmnopqrstuvwxyz'
    for sigla in sorted(grupos.keys()):
        trafos_no_grupo = grupos[sigla]
        if len(trafos_no_grupo) == 1:
            dicionario_mascarado[trafos_no_grupo[0]] = f"T{contador_t}"
        else:
            for i, nome_original in enumerate(trafos_no_grupo):
                dicionario_mascarado[nome_original] = f"T{contador_t}{alfabeto[i]}"
        contador_t += 1
    with open(caminho_mapeamento, 'w', encoding='utf-8') as f:
        json.dump(dicionario_mascarado, f, ensure_ascii=False, indent=4)
else:
    with open(caminho_mapeamento, 'r', encoding='utf-8') as f:
        dicionario_mascarado = json.load(f)

# ETAPA 3: Realizando pré-processamento inicial, ajustando tipo de dados, renomeando, etc
df.rename(columns={'Potência Ativa': 'P', 'Potência Reativa': 'Q', 'Data/Hora Medição': 'datahora', 'Equipamento Medição': 'id'}, inplace=True)
df['id'] = df['id'].map(dicionario_mascarado)
df['P'] = pd.to_numeric(df['P'].str.replace(',', '.'), errors='coerce')
df['Q'] = pd.to_numeric(df['Q'].str.replace(',', '.'), errors='coerce')
df['datahora'] = pd.to_datetime(df['datahora'], format='%d/%m/%Y %H:%M:%S', errors='coerce')
df.dropna(subset=['id', 'datahora', 'P', 'Q'], inplace=True)
df['S'] = np.sqrt(df['P'].abs()**2 + df['Q'].abs()**2)
# df = df[['id', 'datahora', 'S']]

# ETAPA 4: Aplicando filtros de limpeza de outliers
# Corrigir as sequências de zeros
df_etapa_1 = df.groupby('id').apply(lambda group: corrigir_sequencias_de_zeros(group, 'S')).reset_index(drop=True)
# Corrigir os picos isolados
df_limpo = df_etapa_1.groupby('id').apply(lambda group: filtrar_e_imputar_spikes_robusto(group, 'S')).reset_index(drop=True)

# ETAPA 5: Aplicando correção de escala (> 50)
df_limpo.loc[df_limpo['S'] > 50, 'S'] /= 10

# ETAPA 6: Normalizando a demanda (Min-Max scalrr)
df_limpo['S_normalizado'] = df_limpo.groupby('id')['S'].transform(
    lambda x: MinMaxScaler().fit_transform(x.values.reshape(-1, 1)).flatten()
)

# ETAPA 7: Gerando e salvando o dataset final
df_hourly = df_limpo[['id', 'datahora', 'S_normalizado']].copy()
df_hourly.rename(columns={'S_normalizado': 'S'}, inplace=True) # Renomeia para 'S' pra simplificar

caminho_saida_csv = os.path.join(caminho_dados_tratados, 'transformers_dataset.csv')
df_hourly.to_csv(caminho_saida_csv, index=False, sep=';')

#### Gráficos Pré e Pós tratamento 

In [ ]:
import plotly.io as pio
pio.renderers.default = "vscode"

fig_aparente_orig = px.line(df, x='datahora', y='P', color='id',
                       title='Potência Aparente ao Longo do Tempo por Transformador',
                       labels={'S': 'Potência Aparente (kVA)', 'Dia': 'Data'})

fig_aparente_orig.show()

In [ ]:
fig_aparente_trat = px.line(df_hourly, x='datahora', y='S', color='id',
                       title='Potência Aparente ao Longo do Tempo por Transformador - Tratado',
                       labels={'S': 'Potência Aparente (kVA)', 'Dia': 'Data'})

fig_aparente_trat.show()

### Ajustando para pegar a potência máx, min e méd por dia

In [ ]:
df_hourly['datahora'] = pd.to_datetime(df_hourly['datahora'], format='%Y-%m-%d %H:%M:%S')

df_daily = (
    df_hourly
    .set_index('datahora')
    .groupby('id')
    .resample('D')
    .agg(
        Smax=('S', 'max'),
        Smin=('S', 'min'),
        Smean=('S', 'mean')
    )
    .reset_index()
)


df_daily.to_csv(r"G:\Meu Drive\Estudos\Mestrado\Github\masters\bases_tratadas\daily_peak_transformers_dataset.csv", index=False, sep=';')


In [ ]:
df_daily

## Multivariate

In [ ]:
# df_hourly

In [ ]:
# gd = pd.read_csv('G:/Meu Drive/Estudos/Mestrado/Github/masters/bases_originais/EntrantesGD.csv',  sep=';', encoding='latin-1')
# clientes = pd.read_csv('G:/Meu Drive/Estudos/Mestrado/Github/masters/bases_originais/Base_Quantidade_Clientes.csv', sep=';', encoding='latin-1')

# data_inicio = '2018-01-01'
# data_fim = '2024-12-31'
# datas_completas = pd.date_range(start=data_inicio, end=data_fim, freq='D')

In [ ]:
# gd

In [ ]:
# gd.rename(columns={
#     'Tempo': 'datahora',
# }, inplace=True)

# gd['datahora'] = pd.to_datetime(gd['datahora'], format='%Y-%m-%d %H:%M:%S')

# trafos = gd["TRAFO"].unique()
# trafo_datas = pd.DataFrame([(d, t) for d in datas_completas for t in trafos], columns=["datahora", "TRAFO"])
# gd_max = gd.groupby(["datahora", "TRAFO"])['PotenciaAcumulada'].max().reset_index()
# gd = trafo_datas.merge(gd_max, on=["datahora", "TRAFO"], how="left").fillna(0)
# gd

In [ ]:
# clientes['DATA'] = pd.to_datetime(clientes['DATA'])

# # One-hot encoding
# dummies = pd.get_dummies(clientes[['CLASSE', 'DSC_GRUPO_FORNECIMENTO']].astype(str),
#                          prefix=['classe', 'fornec'])

# # Multiplicando as colunas pelo valor de QTD_CLIENTES, pra ser um "peso"
# dummies_mult = dummies.multiply(clientes['QTD_CLIENTES'], axis=0)

# clientes_concat = pd.concat([clientes[['TRAFO', 'DATA']], dummies_mult], axis=1)
# clientes2 = clientes_concat.groupby(['TRAFO', 'DATA']).sum().reset_index()
# clientes2.drop('classe_0', axis=1) # drop classe que n existe

# clientes2.rename(columns={
#     'DATA': 'datahora',
# }, inplace=True)


# horas = pd.DataFrame({
#     'HORA': pd.date_range('00:00', '23:00', freq='H').time
# })

# clientes2 = clientes2.merge(horas, how='cross')

# clientes2['datahora'] = pd.to_datetime(
#     clientes2['datahora'].dt.strftime('%Y-%m-%d') + ' ' + clientes2['HORA'].astype(str)
# )

# clientes2.drop(columns='HORA', inplace=True)

In [ ]:
# gd['id'] = gd['TRAFO'].map(dicionario_mascarado)
# clientes2['id'] = clientes2['TRAFO'].map(dicionario_mascarado)


In [ ]:
# gd = gd[['datahora', 'PotenciaAcumulada', 'id']]
# clientes2 = clientes2[['id', 'datahora',	'classe_0',	'classe_1',	'classe_2',	'classe_3',	'classe_4',	'classe_5',	'classe_6',	'classe_7',	'classe_8',	'fornec_ALTA',	'fornec_BAIXA']]

In [ ]:
# df = pd.merge(df_hourly, gd, on=['datahora', 'id'], how='left')

# df = pd.merge(df, clientes2, on=['datahora', 'id'], how='left')

# data = df.fillna(0)
# data

In [ ]:
# data.to_csv(r"G:\Meu Drive\Estudos\Mestrado\Github\masters\bases_tratadas\full_hourly_transformers_dataset.csv", index=False, sep=';')